In [1]:
import pandas as pd
import numpy as np
import os
import heapq
import scipy.sparse as sparse

In [10]:
class Pre_Processer:
    def __init__(self):
        self.size = 150*10**3
        self.size_name = "medium"
        
    def read_data(self):
        self.data_users = pd.read_excel("Data.xlsx", sheet_name="Users")
        self.data_tracking = pd.read_excel("Data.xlsx", sheet_name="Tracking")
        
    def read_data_twocsv(self):
        self.data_users = pd.read_csv("../StatistaUsers500.csv")
        self.data_tracking = pd.read_csv("../StatistaTracking500.csv")
        
    def read_processed_data(self):
        # Read_Data: load user data from csv.
        self.user_data = pd.read_csv(f"df_user_data_{self.size_name}.csv")
        self.user_data = self.user_data.drop("Unnamed: 0", axis=1)
        
    def read_df_ratings_matrix(self):
        self.matrix_asFrame = pd.read_csv(f"df_ratings_matrix_{self.size_name}.csv")
        self.matrix_asFrame = self.matrix_asFrame.set_index("Unnamed: 0")
        self.matrix_asFrame.index.name = "UserID"
        
    def pre_processing(self):
        print("You are starting with Pre-Processing step 1 here. Otherwise run read_processed_data()")
        
        self.data_tracking = self.data_tracking.drop(labels=[' accessType;;;;;', 'Unnamed: 9',
        'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
        'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
        'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
        'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
        'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
        'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33'], axis=1)
        
        self.data_tracking.columns = ['idUser', 'day', 'month', 'year', 'idContent', 'contentSubType',
               'contentBranch', 'contentName']
    
    def set_ratings_size(self, exponent):
        self.size = 100*10**exponent
        
        if exponent == 0:
            self.size_name = "extra_small"
        elif exponent == 0.5:
            self.size_name = "small"
        elif exponent == 1:
            self.size_name = "medium"
        elif exponent == 2:
            self.size_name = "large"
        elif exponent == 999:
            self.size_name = "full"
        else:
            self.size_name = "different_size"
            
        print(f"Data Frame size set to: {self.size_name}.")
            
        
    def create_ratings_data(self):
        # v2.0 Get all the viewed content for each user
        user_data = pd.DataFrame(columns=["idUser", "idContent", "timesViewed"])
        data_dict = {"idUser": "user", "idContent": "content", "timesViewed": "viewed"}
        user = []
        content = []
        viewed = []

        for i in range(0, len(self.data_tracking["idUser"])):
            if i == self.size:
                break

            # Save & Create new user_list if this row starts a new users data.
            if i != 0 and self.data_tracking["idUser"].iloc[i] != self.data_tracking["idUser"].iloc[i-1]:
                print(data_dict)
                data_dict = {"idUser": user, "idContent": content, "timesViewed": viewed}
                this_user_data = pd.DataFrame.from_dict(data_dict)
                user_data = pd.concat([user_data, this_user_data])
                user = []
                content = []
                viewed = []

            item_found = False

            for j in range(len(content), 0, -1):
                if content[j-1] == self.data_tracking["idContent"].iloc[i]:
                    viewed[j-1] += 1  
                    item_found = True
                    break

            if len(content) == 0 or item_found == False:
                user.append(self.data_tracking["idUser"].iloc[i]) 
                content.append(self.data_tracking["idContent"].iloc[i]) 
                viewed.append(1)

        # save the user data frame to save time
        user_data.to_csv(f"df_user_data_{self.size_name}.csv")   
        
    def create_ratings_dataframe(self):
        print("You are starting with Pre-Processing step 2 here. Otherwise run pre_processing()")
        
        num_contents = 0
        num_users = 0
        id_contents = list()
        id_users = list()
        
        for id in Data.user_data["idContent"]:
            if id not in id_contents:
                id_contents.append(id)
                num_contents += 1
                
        for id in Data.user_data["idUser"]:
            if id not in id_users:
                id_users.append(id)
                num_users += 1
                
        self.matrix_asFrame = pd.DataFrame(index=id_users, columns=id_contents)
        
        # create unary matrix
        """for user in self.matrix_asFrame.index:
            for content in self.user_data[self.user_data["idUser"] == user]["idContent"]:
                    self.matrix_asFrame.loc[user, content] = 1"""
        
        # create numerical matrix with view counts
        for user in self.matrix_asFrame.index:
            for j in range(0, len(self.user_data[self.user_data["idUser"] == user]["idContent"])):
                    content = self.user_data[self.user_data["idUser"] == user].iloc[j]["idContent"]
                    views = self.user_data[self.user_data["idUser"] == user].iloc[j]["timesViewed"]
                    self.matrix_asFrame.loc[user, content] = views

        self.matrix_asFrame = self.matrix_asFrame.fillna(0)
        
        # save the user data frame to save time
        self.matrix_asFrame.to_csv(f"df_ratings_matrix_{self.size_name}.csv") 

    def create_ratings_matrix(self):
        print("You are starting with Pre-Processing step 3 here. Otherwise run pre_processing()")

        self.ratings_matrix = self.matrix_asFrame.values
        np.save(f"ratings_matrix_{self.size_name}", self.ratings_matrix)
    
    def create_sparse_ratings_matrix(self):
        print("You are starting with Pre-Processing step 3 here. Otherwise run pre_processing()")
        
        self.sparse_matrix = sparse.csr_matrix(self.matrix_asFrame.values)
        sparse.save_npz(f"ratings_matrix_{self.size_name}", self.sparse_matrix, compressed=True)

In [11]:
Data = Pre_Processer()

In [14]:
# Full pre-processing step 1. Optional: set_ratings_size(). If not set 150*10**3 = 150.000 rows of tracking data will be read.
Data.read_data_twocsv()
#Data.pre_processing()
#Data.set_ratings_size(999)
#Data.create_ratings_data()

In [ ]:
# If Pre-Processing Step 1 is already done.
Data.set_ratings_size(999)
Data.read_processed_data()
Data.create_ratings_dataframe()

In [ ]:
# If Pre-Processing Step 1 & 2 are already done.
Data.set_ratings_size(999)
Data.read_df_ratings_matrix()
#Data.create_ratings_matrix()
Data.create_sparse_ratings_matrix()

In [7]:
Data.matrix_asFrame.head(10)

""
UserID


In [62]:
Data.matrix_asFrame.mean().nlargest(10)

0           0.262045
264810      0.024677
272014      0.021152
243         0.019976
200         0.017626
3979        0.011751
70000000    0.011751
270291      0.011751
271496      0.011751
276623      0.011751
dtype: float64

In [65]:
0.024677 * 851

21.000127

In [67]:
Data.matrix_asFrame["264810"][Data.matrix_asFrame["264810"] > 0]

UserID
323811     1
510587     1
580991     1
779764     1
993682     1
1037458    1
1062591    1
1066778    1
1244472    1
1307724    1
1328238    1
1490970    1
1571724    1
1614781    1
1668070    1
1771680    1
1855214    1
1890625    1
1943733    1
1980007    1
1992377    1
Name: 264810, dtype: int64

In [16]:
Data.data_tracking.tail()

,idUser,day,month,year,idContent,contentSubType,contentBranch,contentName,accessType
1618797,228776,24,8,2015,0,home,NaN,home,view
1618798,228776,25,8,2015,0,home,NaN,home,view
1618799,228776,22,10,2015,0,home,NaN,home,view
1618800,228776,27,10,2015,0,home,NaN,home,view
1618801,331436,21,8,2013,236823,Statistic,978,Prevalence of obesity among adults in selected...,view


In [8]:
Data.read_data_twocsv()

In [17]:
1618801/500

3237.602